In [1]:
import sys
import pandas as pd
import re
import os
import math
import gzip
import matplotlib.pyplot as plt
import numpy as np


In [2]:
def loadFasta(fasta_path, as_dict=False,uppercase=False, stop_at=None,
              revcomp=False):
    fastas = []
    seq = None
    header = None
    for r in (gzip.open(fasta_path) if fasta_path.endswith(".gz") else open(fasta_path)):
        if type(r) is bytes: r = r.decode("utf-8")
        r = r.strip()
        if r.startswith(">"):
            if seq != None and header != None:
                fastas.append([header, seq])
                if stop_at != None and len(fastas) >= stop_at:
                    break
            seq = ""
            header = r[1:]
        else:
            if seq != None:
                seq += r.upper() if uppercase else r
            else:
                seq = r.upper() if uppercase else r
    if stop_at != None and len(fastas) < stop_at:
        fastas.append([header, seq])
    elif stop_at == None:
        fastas.append([header, seq])
    if as_dict:
        return {h: s for h, s in fastas}
    if(revcomp):
        for rec in fastas:
            rc=generate_complementary_sequence(rec[1])
            rec[1]=rec[1]+"NNNNNNNNNNNNNNNNNNNN"+rc
    return pd.DataFrame({'location': [e[0].split('=')[-1] for e in fastas],
                         'sequence': [e[1] for e in fastas]})

In [3]:
# insert GR and AP1 to low activation sequences at varying distances
# output format is for prediction
forward_ap1 = 'TGAGTCAT'
forward_gr  = 'GAACATTATGTTC'

def insert_motifs_pred(df, seq_len=300):
    seq = df['sequence']
    loc = df['location']
    ls_distance = []
    ls_ref = []
    ls_allele = []
    ls_seq = []

    for i in range(len(df)):
        sequence = seq[i]
        max_dist = seq_len - len(forward_gr)
        dist = np.random.randint(len(forward_ap1), max_dist + 1)  # dist: from begining of a motif to the begining of another motif
        
        if dist == max_dist: pos_ap1 = 0
        else: pos_ap1 = np.random.randint(0, seq_len - len(forward_gr) - dist)
        pos_gr = pos_ap1 + dist

        sequence = sequence[:pos_ap1] + forward_ap1 + sequence[pos_ap1 + len(forward_ap1):]
        sequence = sequence[:pos_gr] + forward_gr + sequence[pos_gr + len(forward_gr):]
        
        ls_seq.append(sequence)
        ls_distance.append(dist)
        ls_ref.append('ref='+sequence[seq_len//2])
        ls_allele.append(sequence[seq_len//2])
        
    installed = pd.DataFrame({'location':df['location'],'ref':ls_ref,'allele':ls_allele,'sequence':ls_seq})
    loc_dist = pd.DataFrame({'location':df['location'],'distance':ls_distance})

    return installed, loc_dist

In [6]:
def get_low_act_5000(in_dir, out_dir):
    # get the 5000 lowest activation sequences from test set of Dex
    # abs(median(log(Dex))-log(Dex)) close to 0
    counts = pd.read_csv(in_dir+'/test-counts.txt.gz', compression="gzip", sep='\t', skiprows=1, header=None)
    counts.columns = ['DNA1','DNA2','DNA3','DNA4','DNA5','RNA1','RNA2','RNA3','RNA4']
    counts['DNA_aver'] = (counts['DNA1']+counts['DNA1']+counts['DNA1']+counts['DNA1']+counts['DNA1'])/5
    counts['RNA_aver'] = (counts['RNA1']+counts['RNA2']+counts['RNA3']+counts['RNA4'])/4
    counts['theta'] = np.log(counts['RNA_aver']/counts['DNA_aver'])
    counts['activation'] = abs(counts['theta'].median()-counts['theta'])
    counts.sort_values(by='activation',inplace=True)
    counts = counts[:5000]
    fasta = loadFasta(in_dir+'/test.fasta.gz')
    fasta = fasta.loc[counts.index]
    fasta.reset_index(drop=True, inplace=True)
    fasta.to_csv(out_dir+'/low_act_5000.txt',sep='\t',header=None,index=False)

    # insert motifs at varying distances
    installed, loc_dist = insert_motifs_pred(fasta)
    installed.to_csv(out_dir+'/low_act_5000_installed.txt', sep='\t',header=None,index=False)
    loc_dist.to_csv(out_dir+'/low_act_5000_installed_dist.txt', sep='\t',header=None,index=False)

In [7]:
get_low_act_5000('/hpc/group/igvf/A549/GR-AP1/simulated-seq/data/Dex-200', '/hpc/group/igvf/A549/GR-AP1/simulated-seq/data/Dex-200')

In [6]:
all_train = loadFasta('/datacommons/igvf-pm/A549/full-set/Dex-200/300-bases/data-normalized/train.fasta.gz')
# get 5000 low-activation sequence from training set of Dex: abs(median(log(Dex))-log(Dex)) close to 0 from experimental counts
counts = pd.read_csv('/datacommons/igvf-pm/A549/full-set/Dex-200/300-bases/data-normalized/train-counts.txt.gz', compression="gzip", sep='\t', skiprows=1, header=None)
counts.columns = ['DNA1','DNA2','DNA3','DNA4','DNA5','RNA1','RNA2','RNA3','RNA4']
counts['DNA_aver'] = (counts['DNA1']+counts['DNA1']+counts['DNA1']+counts['DNA1']+counts['DNA1'])/5
counts['RNA_aver'] = (counts['RNA1']+counts['RNA2']+counts['RNA3']+counts['RNA4'])/4
counts['theta'] = np.log(counts['RNA_aver']/counts['DNA_aver'])
counts['activation'] = abs(counts['theta'].median()-counts['theta'])
counts.sort_values(by='activation',inplace=True)
counts = counts[:5000]
counts

,DNA1,DNA2,DNA3,DNA4,DNA5,RNA1,RNA2,RNA3,RNA4,DNA_aver,RNA_aver,theta,activation
1366065,0.998980,1.064236,1.050659,0.978319,1.056019,1.040806,3.053206,1.812080,0.894977,0.998980,1.700267,0.531806,1.519404e-07
447648,1.382367,1.356350,1.190268,1.437359,1.470051,2.671450,2.024191,2.982936,1.732599,1.382367,2.352794,0.531806,1.519404e-07
715169,1.168348,1.247945,1.234613,1.197423,1.137119,1.930261,1.363602,1.965881,2.694383,1.168348,1.988532,0.531806,2.630957e-07
93294,1.481037,1.545088,1.491052,1.604272,1.503496,2.305634,3.074418,2.758016,1.944841,1.481037,2.520727,0.531805,1.217074e-06
530535,1.261267,1.122617,1.420087,1.392818,1.342155,2.762899,2.507008,1.795506,1.521303,1.261267,2.146679,0.531805,1.351904e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049252,1.339384,1.452326,1.471403,1.308361,1.280040,2.051518,3.038055,1.405221,2.640495,1.339384,2.283822,0.533640,1.834194e-03
773396,1.186309,1.224786,1.158240,1.339675,1.150456,2.263488,1.130021,2.752427,1.945314,1.186309,2.022812,0.533642,1.835532e-03
354826,8.507863,8.518713,8.679517,9.032385,9.263957,24.750328,12.423927,9.887226,10.966610,8.507863,14.507023,0.533642,1.835852e-03
1628904,1.075799,1.220727,1.211549,1.146359,1.067875,1.113617,1.707532,3.033037,1.483329,1.075799,1.834379,0.533642,1.835966e-03


In [7]:
low_act_seq = all_train.loc[counts.index]
low_act_seq.to_csv('/datacommons/igvf-pm/A549/GR-AP1/simulated-seq/data/Dex-200/diluting/low_act_5000.txt', sep='\t', header=None, index=False)
low_act_seq

,location,sequence
1366065,chr3:71940000-71940300,CTACTGAGAGGCCCTTTAGCTAATTGAAAGGAGAAAGTGGTTCCAT...
447648,chr15:98665350-98665650,TGCATCTGCCTCCGTTTGAAGCATCCTCCTCCAGTAAGTGGGTCAT...
715169,chr10:2105300-2105600,TGTTCCGGATGGTGAATCCTCACGAGTCTGCAGCAACCTCAATTCT...
93294,chr19:10350700-10351000,AGACAGGAGTAAGGCACACGGTGTGGGTGAGGCTGACATCCCCCTC...
530535,chr13:99978000-99978300,ATGAACACTGGCCCCCGCGGCCCTGAGCTGGATCCGCTCAAGCCCC...
...,...,...
1049252,chr7:55952550-55952850,ATGGTGAAACTCTGTCTTTACTAAAAATACAAAAATTAGCTCGGCG...
773396,chr10:102655350-102655650,GGGCCATTTCCAAATTGAGATCTGCAGTTTCCTGCACCTGTAGCTC...
354826,chr16:46390650-46390950,AATCATTATCGAATGGAATCGAATGGAATCATCGAATGGAATTGAT...
1628904,chr1:91737800-91738100,AGACGGTGTCGGGACAAGATTCAAGGTCTCCAGGTATGGTCTGAGA...


In [8]:
# exclude 5000 low-activation sequences from whole training set to select remaining sequences
rdm_seq = all_train.loc[~all_train.index.isin(counts.index)]
rdm_seq.to_csv('/datacommons/igvf-pm/A549/GR-AP1/simulated-seq/data/Dex-200/diluting/rdm_seq.txt', sep='\t', header=None, index=False)
rdm_seq

,location,sequence
0,chr22:10716750-10717050,TAACATGTCACGTACTATAAGGTTTGCATGGCATGGTGGCAGCTGA...
1,chr22:10716850-10717150,TTTTTCCTTAAACATTTGAATCCATTCAATGTCATTCCATTCCAAT...
2,chr22:10716950-10717250,TCAATCCCATTAAATTCTACTCCATTCCATTCTTTTCCATTCCATT...
3,chr22:10717000-10717300,TCCATTTGAACCCAATGAATTCCACTGCATTCAATTGAAGTCCATT...
4,chr22:10717050-10717350,TCCATTCGGGTCCATTCCCTTCAATTCCATTCGTGTCAATTCCATT...
...,...,...
1700565,chr1:248934500-248934800,GGGGAATAGAAGAAAGAAAGAAAAGGAAGTTGCCCAGGGATAGTTA...
1700566,chr1:248934650-248934950,ACACACACACACACACACACACACACACACACACATATAAAAATAG...
1700567,chr1:248934700-248935000,TAGTTGTGACTTTATAATCTTTGAGGAAGAACTTTCCTCAAAGTTT...
1700568,chr1:248934750-248935050,CAGTGCTTTGTAAGCATTGTCTCCATAAAAGTCAACCTTACTTCCT...


In [23]:
# get 5000 low-activation sequences for biased downsampling experiment, since train/test/validation were resplit
test = loadFasta('/datacommons/igvf-pm/A549/GR-AP1/simulated-seq/data/Dex-200-biased/test.fasta.gz')
all = loadFasta('/datacommons/igvf-pm/A549/processed-data/300-bases/Dex-200/Dex-200-all.fasta.gz')
all_counts = pd.read_csv('/datacommons/igvf-pm/A549/processed-data/300-bases/Dex-200/Dex-200-all-normalized-counts.txt.gz', compression="gzip", sep='\t', skiprows=1, header=None)
counts_idx = list(pd.merge(test, all, how='inner', on='location').index)
counts = all_counts.loc[all_counts.index.isin(counts_idx)]

counts = counts.reset_index(drop=True)
counts.columns = ['DNA1','DNA2','DNA3','DNA4','DNA5','RNA1','RNA2','RNA3','RNA4']
counts['DNA_aver'] = (counts['DNA1']+counts['DNA1']+counts['DNA1']+counts['DNA1']+counts['DNA1'])/5
counts['RNA_aver'] = (counts['RNA1']+counts['RNA2']+counts['RNA3']+counts['RNA4'])/4
counts['theta'] = np.log(counts['RNA_aver']/counts['DNA_aver'])
counts['activation'] = abs(counts['theta'].median()-counts['theta'])
counts = counts.sort_values(by='activation')
counts = counts[:5000]

test = test.loc[test.index.isin(counts.index)]
test = test.reset_index(drop=True)

installed, loc_dist = insert_motifs_pred(test)

installed.to_csv('/datacommons/igvf-pm/A549/GR-AP1/simulated-seq/data/Dex-200-biased/low_act_5000_test.txt', sep='\t', header=None, index=False)
loc_dist.to_csv('/datacommons/igvf-pm/A549/GR-AP1/simulated-seq/data/Dex-200-biased/low_act_5000_test_dist.txt', sep='\t', header=None, index=False)

In [24]:
installed

,location,ref,allele,sequence
0,chr22:10723400-10723700,ref=C,C,CATTCCATTCCATTCCATTAAATTCCATTCTGAGTCATTCCATTCC...
1,chr22:11435650-11435950,ref=T,T,GGTGGGGCTCCTGCCTGCCAATTTAGAAGGGGTGGGGCTCCCACCT...
2,chr22:17104800-17105100,ref=G,G,GACACTCCAGGTAGGGGACATGCGGCTGTCCTAGGCCATACTGGGA...
3,chr22:17591750-17592050,ref=C,C,ACCCGGGAGGCTGAGTCATCAGTGAGCCGAGATGGCACCACTGCAC...
4,chr22:18131200-18131500,ref=A,A,GAGAACAGAACACTCTCCCCGCCCCAGCCTGATTCCTGCCTTACCC...
...,...,...,...,...
4995,chr1:242519700-242520000,ref=A,A,TGAGTCATGAGCAAGCAAGGGCCCCACTAGCCAAGAAGAGTCAGGC...
4996,chr1:242754400-242754700,ref=A,A,ATGTCAGAAAGTGCCTGGAAGTAGGGAGTGGTCTAAAGAATTTGCT...
4997,chr1:242882150-242882450,ref=G,G,GCCTGCTCTGAGAAGACACCATGCAGCGTCTTCGTGTCTTGGGCCA...
4998,chr1:245894050-245894350,ref=G,G,TCTGAGTCATTGAAGCCAGCTGGACTTCCAGGGTGGAGTGGGGAAT...


In [25]:
loc_dist

,location,distance
0,chr22:10723400-10723700,201
1,chr22:11435650-11435950,10
2,chr22:17104800-17105100,28
3,chr22:17591750-17592050,183
4,chr22:18131200-18131500,95
...,...,...
4995,chr1:242519700-242520000,272
4996,chr1:242754400-242754700,149
4997,chr1:242882150-242882450,37
4998,chr1:245894050-245894350,282
